# Zadanie 5 – Test modeli i wersji promptów systemowych
Testujemy różne modele Ollama oraz dwa style promptów do generowania broszury firmowej.

In [ ]:
import requests
from bs4 import BeautifulSoup
import ollama
from IPython.display import Markdown, display

In [ ]:
def get_page_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

In [ ]:
system_prompt_1 = "Jesteś ekspertem marketingu. Napisz profesjonalną broszurę firmy na podstawie treści strony."
system_prompt_2 = "Napisz lekką i przyjazną broszurę o firmie. Użyj przystępnego języka. Odpowiedź w markdown."

In [ ]:
def create_brochure_prompt(text):
    return f"Oto treść strony firmy. Napisz na jej podstawie broszurę: {text[:4000]}"

In [ ]:
def generate_brochure(url, model='llama3.2', prompt_version=1):
    text = get_page_text(url)
    system_prompt = system_prompt_1 if prompt_version == 1 else system_prompt_2
    prompt = create_brochure_prompt(text)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]
    response = ollama.chat(model=model, messages=messages)
    return response['message']['content']

In [ ]:
# Przykład testu na różnych modelach i promptach
models = ['llama3.2', 'llama3.3', 'deepseek-r1:8b', 'gemma2:27b']
urls = ['https://pl.wikipedia.org/wiki/OpenAI', 'https://www.anaconda.com']

for url in urls:
    for model in models:
        for version in [1, 2]:
            print(f"\nModel: {model} | URL: {url} | Prompt wersja: {version}\n")
            try:
                result = generate_brochure(url, model=model, prompt_version=version)
                display(Markdown(result))
            except Exception as e:
                print(f"Błąd przy {model}, wersja {version}: {e}")